# Tobig's 14기 2주차 Optimization 과제
### Made by 이지용

# Gradient Descent 구현하기

### 1) "..." 표시되어 있는 빈 칸을 채워주세요  
### 2) 강의내용과 코드에 대해 공부한 내용을 적어서 과제를 채워주세요

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
data = pd.read_csv('assignment_2.csv')
data.head()

,Label,bias,experience,salary
0,1,1,0.7,48000
1,0,1,1.9,48000
2,1,1,2.5,60000
3,0,1,4.2,63000
4,0,1,6.0,76000


## Train Test 데이터 나누기
### 데이터셋을 train/test로 나눠주는 메소드  
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], test_size=0.25, random_state = 0)

In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((150, 3), (50, 3), (150,), (50,))

## Scaling  

experience와 salary의 단위, 평균, 분산이 크게 차이나므로 scaler를 사용해 단위를 맞춰줍니다. 

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
bias_train = X_train["bias"]
bias_train = bias_train.reset_index()["bias"]
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_train["bias"] = bias_train
X_train.head()

,bias,experience,salary
0,1,0.187893,-1.143335
1,1,1.185555,0.043974
2,1,-0.310938,-0.351795
3,1,-1.629277,-1.341220
4,1,-1.308600,0.043974


이때 scaler는 X_train에 fit 해주시고, fit한 scaler를 X_test에 적용시켜줍니다.  
똑같이 X_test에다 fit하면 안돼요!

In [7]:
bias_test = X_test["bias"]
bias_test = bias_test.reset_index()["bias"]
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
X_test["bias"] = bias_test
X_test.head()

,bias,experience,salary
0,1,-1.344231,-0.615642
1,1,0.508570,0.307821
2,1,-0.310938,0.571667
3,1,1.363709,1.956862
4,1,-0.987923,-0.747565


In [8]:
# parameter 개수
N = len(X_train.loc[0])

In [9]:
# 초기 parameter들을 임의로 설정해줍니다.
parameters = np.array([random.random() for i in range(N)])
parameters

array([0.54443325, 0.26162818, 0.51282689])

### * LaTeX   

Jupyter Notebook은 LaTeX 문법으로 수식 입력을 지원하고 있습니다.  
http://triki.net/apps/3466  
https://jjycjnmath.tistory.com/117

## Logistic Function

## $p = \frac{1}{1 +exp(-X_{i}\theta)}$

In [10]:
def logistic(X, parameters):
    z = 0
    for i in range(len(parameters)) :
        z += X[i]*parameters[i]
    p = 1 / (1 + np.exp(-z))
    
    return p

In [11]:
logistic(X_train.iloc[1], parameters)

0.7062331837349426

## Object Function

Object Function : 목적함수는 Gradient Descent를 통해 최적화 하고자 하는 함수입니다.  
로지스틱 회귀의 목적함수를 작성해주세요
## $l(p) =y_{i}logp(X_{i}) + (1-y_{i})log(1-p(X_{i}))$

In [12]:
def minus_log_cross_entropy_i(X, y, parameters) :
    p = logistic(X, parameters)                            # 위에서 작성한 함수를 활용하세요
    loss = -(y*np.log(p) + (1-y)*np.log(1-p))
    return loss

In [13]:
def minus_log_cross_entropy(X_set, y_set, parameters) :
    loss = 0
    for i in range(X_set.shape[0]):
        X = X_set.iloc[i, :]
        y = y_set.iloc[i]
        loss += minus_log_cross_entropy_i(X,y,parameters)
    return loss

In [14]:
minus_log_cross_entropy(X_test, y_test, parameters)

54.16437514564639

## Gradient of Minus Log Cross Entropy

## ${\partial\over{\partial \theta_j}}l(p)= -\Sigma(y_{i}-p_{i})x_{ij}$

In [15]:
# cross_entropy를 theta_j에 대해 미분한 값을 구하는 함수
def get_gradient_ij_minus_log_cross_entropy(X, y, parameters, j):
    p = logistic(X, parameters)
    gradient = -((y-p)*X[j])
    return gradient

In [16]:
get_gradient_ij_minus_log_cross_entropy(X_train.iloc[0, :], y_train.iloc[0], parameters, 1)

-0.09360556881569491

## Batch Gradient Descent  

Batch Gradient Descent : 학습 한 번에 모든 데이터셋에 대해 기울기를 구한다

In [17]:
def get_gradients_bgd(X_train, y_train, parameters) :
    gradients = [0 for i in range(len(parameters))]
    
    for i in range(len(y_train)):
        X = X_train.iloc[i, :]
        y = y_train.iloc[i]
        for j in range(len(parameters)):
            gradients[j] += get_gradient_ij_minus_log_cross_entropy(X,y,parameters,j)
            
    return gradients

In [18]:
gradients_bgd = get_gradients_bgd(X_train, y_train, parameters)
gradients_bgd

[50.67112203401287, 2.4219245570325048, 36.338210245820136]

## Stochastic Gradient Descent  

Stochastic Gradient Descent : 학습 한 번에 임의의 데이터에 대해서만 기울기를 구한다

In [19]:
def get_gradients_sgd(X_train, y_train, parameters) :
    gradients = [0 for i in range(len(parameters))]
    r = int(random.random()*len(y_train))
    X = X_train.iloc[r, :]
    y = y_train.iloc[r]
        
    for j in range(len(parameters)):
        gradients[j] = get_gradient_ij_minus_log_cross_entropy(X,y,parameters,j)
        
    return gradients

In [20]:
gradients_sgd = get_gradients_sgd(X_train, y_train, parameters)
gradients_sgd

[0.6700257013609217, 0.10201962036332625, 0.16205195883202325]

## Update Parameters  

In [21]:
def update_parameters(parameters, gradients, learning_rate) :
    for i in range(len(parameters)) :
        gradients[i] *= learning_rate
    parameters -= gradients
    return parameters

In [22]:
update_parameters(parameters, gradients_bgd, 0.01)

array([0.03772203, 0.23740893, 0.14944479])

## Gradient Descent  

위에서 작성한 함수들을 조합해서 Gradient Descent를 진행하는 함수를 완성해주세요

learning_rate = 학습률  
max_iter = 최대 반복 횟수  
tolerance = Step이 너무 작아서 더 이상의 학습이 무의미할 때 학습을 멈추는 조건

In [23]:
def gradient_descent(X_train, y_train, learning_rate=0.01, max_iter=100000, tolerance=0.0001, optimizer="bgd") :
    count = 1
    point = 100 if optimizer == "bgd" else 10000
    N = len(X_train.iloc[0])
    parameters = np.array([random.random() for i in range(N)])
    gradients = [0 for i in range(N)]
    loss = minus_log_cross_entropy(X_train, y_train, parameters)
    
    while count < max_iter :
        
        if optimizer == "bgd" :
            gradients = get_gradients_bgd(X_train, y_train, parameters)
        elif optimizer == "sgd" :
            gradients = get_gradients_sgd(X_train, y_train, parameters)
            # loss, 중단 확인
        if count%point == 0 :
            new_loss = minus_log_cross_entropy(X_train, y_train, parameters)
            print(count, "loss: ",new_loss, "params: ", parameters, "gradients: ", gradients)
            
            #중단 조건
            if abs(new_loss-loss) < tolerance*len(y_train) :
                break
            loss = new_loss
                
            
                
        parameters = update_parameters(parameters, gradients, learning_rate)
        count += 1
    return parameters

In [24]:
new_param_bgd = gradient_descent(X_train, y_train)
new_param_bgd

100 loss:  45.42437339567176 params:  [-1.61714625  3.45219219 -3.27993607] gradients:  [0.28659373441759584, -0.9437390075205173, 0.8825707910726842]
200 loss:  44.804001368083924 params:  [-1.77992397  3.98445054 -3.77496057] gradients:  [0.08555912091219049, -0.27806305256307406, 0.25719108572326355]
300 loss:  44.74070176664928 params:  [-1.8333487   4.15780041 -3.93502825] gradients:  [0.03073447188863794, -0.09956881608729803, 0.09178293639371582]
400 loss:  44.73219171686312 params:  [-1.85308328  4.22169895 -3.99389445] gradients:  [0.0116735623379422, -0.03777723818530313, 0.03478059216566189]


array([-1.85308328,  4.22169895, -3.99389445])

## Hyper Parameter Tuning

Hyper Parameter들을 매번 다르게 해서 학습을 진행해 보세요. 다른 점들을 발견할 수 있습니다.

In [25]:
new_param_sgd = gradient_descent(X_train, y_train, learning_rate=0.01, max_iter=100000, tolerance=0.0001, optimizer="sgd")
new_param_sgd

10000 loss:  46.47971389731845 params:  [-1.51482358  3.02992877 -2.88366296] gradients:  [-0.09546280627783843, -0.1165778421806262, 0.0020989608338231143]
20000 loss:  45.05699277555738 params:  [-1.6911601   3.71275723 -3.4937813 ] gradients:  [0.557578425246645, -0.03430361872205396, -0.34326864059700807]
30000 loss:  44.91453455297427 params:  [-1.70909511  4.01825492 -3.71012109] gradients:  [-0.9748579602260486, 0.7894098754195691, 0.34295051768746065]
40000 loss:  44.78739335825057 params:  [-1.82063911  4.11485695 -3.97708531] gradients:  [0.015219310308621507, 0.0006904982000465584, 0.009704275283046708]
50000 loss:  45.061979903768005 params:  [-1.81073548  4.10791946 -4.10793537] gradients:  [-0.3216420677858355, 0.2948368995489533, 0.49504083357787165]
60000 loss:  44.758988781536594 params:  [-1.81180718  4.20514528 -4.02334068] gradients:  [0.08845999669472748, 0.07650711214526075, 0.09141446345555959]
70000 loss:  44.75372111263354 params:  [-1.87228303  4.28790063 -3.9

array([-1.87228303,  4.28790063, -3.99658267])

## Predict Label

In [26]:
y_predict = []
for i in range(len(y_test)):
    p = logistic(X_test.iloc[i,:], new_param_bgd)
    if p> 0.5 :
        y_predict.append(1)
    else :
        y_predict.append(0)

## Confusion Matrix

In [27]:
from sklearn.metrics import *
tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
confusion_matrix(y_test, y_predict)

array([[38,  2],
       [ 1,  9]], dtype=int64)

In [28]:
accuracy = (tp+tn) / (tp+fn+fp+tn)
print("accuracy:",accuracy)

accuracy: 0.94
